Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-25 13:52:43--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-25 13:52:44--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  78.0KB/s    in 0.8s    

2021-10-25 13:52:46 (78.0 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [2]:
!gdown https://drive.google.com/uc?id=1K8R_N8BZ4K1coSxAeR5a_0L9BGD-rbAr

Downloading...
From: https://drive.google.com/uc?id=1K8R_N8BZ4K1coSxAeR5a_0L9BGD-rbAr
To: /content/2021SIY7558_B_model.zip
100% 406M/406M [00:02<00:00, 150MB/s]


In [3]:
!unzip 2021SIY7558_B_model.zip

Archive:  2021SIY7558_B_model.zip
  inflating: my-bert-pretrained/config.json  
  inflating: my-bert-pretrained/pytorch_model.bin  
  inflating: my-bert-pretrained/special_tokens_map.json  
  inflating: my-bert-pretrained/tokenizer.json  
  inflating: my-bert-pretrained/tokenizer_config.json  
  inflating: my-bert-pretrained/training_args.bin  
  inflating: my-bert-pretrained/vocab.txt  


In [4]:
!pip install datasets

     |████████████████████████████████| 290 kB 5.2 MB/s 
     |████████████████████████████████| 243 kB 43.9 MB/s 
     |████████████████████████████████| 1.3 MB 44.2 MB/s 
     |████████████████████████████████| 125 kB 44.2 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 160 kB 59.7 MB/s 
     |████████████████████████████████| 271 kB 60.1 MB/s 


In [5]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 895 kB 60.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import pandas as pd
import numpy as np
import csv
import torch
from transformers import Trainer
import re
from datasets import load_metric,load_dataset
from transformers import BertTokenizerFast,BertModel, BertConfig,BertForSequenceClassification,BertForMaskedLM,BertLMHeadModel,DistilBertModel,AutoModelForSequenceClassification
from transformers import pipeline,DataCollatorWithPadding

In [7]:
tokenizer = BertTokenizerFast.from_pretrained('./my-bert-pretrained',padding=True,truncation=True)

In [8]:
data_collator=DataCollatorWithPadding(tokenizer)

In [9]:
def preprocess(sen):
    sen=re.sub("\W"," ",sen)
    sen=re.sub("\s+"," ",sen)
    return sen.lower().strip()

In [10]:
dfx = pd.read_csv('./test.data.txt', delimiter = "\t",header=None)
dfx['text1']=dfx[3]
dfx['text2']=dfx[4]
dftx=dfx[['text1','text2']]
dftx=np.array(dftx)
with open('./formatted.csv','w') as f:
  writer =csv.writer(f)
  writer.writerow(['text1','text2','label'])
  writer.writerows(dftx)
dftx=load_dataset('csv', data_files='./formatted.csv')

Using custom data configuration default-87e8cc29136c5667


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-87e8cc29136c5667/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
def tokenize_func(row):
  return tokenizer(row['text1'],row['text2'],padding=True,truncation=True)
tokenized_data=dftx.map(tokenize_func,batched=True)
tokenized_data = tokenized_data.map(lambda example: {}, remove_columns=['text1','text2'])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/638 [00:00<?, ?ex/s]

In [12]:
model =BertForSequenceClassification.from_pretrained('./my-bert-pretrained')

In [13]:
tokenized_data['train']

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 638
})

In [14]:
trainer = Trainer(
    model=model
)

In [15]:
op=trainer.predict(test_dataset=tokenized_data['train'])

***** Running Prediction *****
  Num examples = 638
  Batch size = 8


In [16]:
len(op)

3

In [17]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [18]:
predictions=np.argmin(op.predictions,axis=-1)

In [19]:
predictions

array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,

In [20]:
with open('output.txt','w') as o:
  for pred in predictions:
    k='T'
    if pred==0:
      k='F'
    o.write(k + "\n")

In [21]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-25 13:55:39--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-25 13:55:40--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-25 13:55:41 (46.8 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.7084639498432602
